# F I R S T - O R D E R - M A M L

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import copy

import gymnasium as gym


### D E V I C E 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')


Device: cuda


### H E L P E R 

In [ ]:
def safe_tensor(x):
    
    return x if torch.is_tensor(x) else torch.tensor(x, dtype = torch.float32).to(device)


### L O G G I N G

In [ ]:
writer = SummaryWriter(log_dir = './runs/FOMAML')


### M E T A - E N V

In [ ]:
class meta_car_continuous:
    
    def __init__(self):
        
        self.base_env = gym.make('MountainCarContinuous-v0')
        self.org_gravity = 0.0025
        self.org_goal_position = 0.45
        self.tasks = []
        self.current_task = (self.org_goal_position, self.org_gravity)
        
    def sample_task(self, num_tasks):
        
        self.tasks = []
        
        for _ in range(num_tasks):
            goal_position = np.random.uniform(0.45, 0.55)
            gravity = np.random.uniform(0.0025, 0.006)
            self.tasks.append((goal_position, gravity))
            
        return self.tasks

    def set_task(self, task):
        
        self.current_task = task
        
        self.base_env.env.gravity = task[1]
        self.base_env.env.goal_position = task[0]

    def reset(self):
        
        obs = self.base_env.reset()
        
        if isinstance(obs, tuple):
            
            obs = obs[0]
            
        return obs

    def step(self, action):
        
        step_output = self.base_env.step(action)
        
        if len(step_output) == 5: 
            
            next_obs, reward, terminated, truncated, info = step_output
            done = terminated or truncated
            
        else:
            
            next_obs, reward, done, info = step_output
            
        return next_obs, reward, done, info

    def get_number(self):
        
        state_dim = self.base_env.observation_space.shape[0]
        action_dim = self.base_env.action_space.shape[0]
        max_action = float(self.base_env.action_space.high[0])
        reward_dim = 1
        
        return state_dim, action_dim, max_action, reward_dim

    def close(self):
        
        self.base_env.close()


### S E T U P 

In [6]:
# Initiate env

env = meta_car_continuous()

tasks = env.sample_task(2)

for task in tasks:
    
    env.set_task(task)

    obs = env.reset()

    print(f'obs: {obs.shape}')

# get number

state_dim, action_dim, max_action, reward_dim = env.get_number()

print(f'state dim: {state_dim} | action dim: {action_dim} | max action: {max_action} | reward dim: {reward_dim}')


obs: (2,)
obs: (2,)
state dim: 2 | action dim: 1 | max action: 1.0 | reward dim: 1


### A S S E M B L Y 

In [ ]:
head_1 = 32
head_2 = 64
head_3 = 64
head_4 = 32

hidden_size = 32
hidden_size_2 = 64


### H Y P E R - X 

In [8]:
class hyper_x(nn.Module):
    
    def __init__(self, state_dim = state_dim, hidden_size = hidden_size, hidden_size_2 = hidden_size_2):
        super(hyper_x, self).__init__()
        
        # input dim
        
        input_dim = state_dim
        
        # mlp
        
        self.hyper = nn.Sequential(
            
            nn.Linear(input_dim, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.SiLU(),
            
            nn.Linear(hidden_size, hidden_size_2),
            nn.LayerNorm(hidden_size_2),
            nn.SiLU(),
            
            
            nn.Linear(hidden_size_2, hidden_size_2),
            nn.LayerNorm(hidden_size_2),
            nn.SiLU(),
            
            
            nn.Linear(hidden_size_2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.SiLU(),
            
        )
        
    def forward(self, state):
        
        hyper = self.hyper(state)
        
        return hyper
    

### P O L I C Y

In [9]:
class policy_net(nn.Module):
    
    def __init__(self, head_1 = head_1, head_2 = head_2, head_3 = head_3, head_4 = head_4, action_dim = action_dim, max_action = max_action):
        super(policy_net, self).__init__()
        
        # max action
        
        self.max_action = max_action
        
        # hyper
        
        self.hyper = hyper_x()
        
        # mlp : [ input -> state -> hyper -> output -> hidden  [ 64 ] -> input -> policy -> head 1 [ 64 ] -> output -> action ]
        
        self.policy = nn.Sequential(
            
            nn.Linear(head_1, head_2),
            nn.LayerNorm(head_2),
            nn.SiLU(),
            
            nn.Linear(head_2, head_3),
            nn.LayerNorm(head_3),
            nn.SiLU(),
            
            nn.Linear(head_3, head_4),
            nn.LayerNorm(head_4),
            nn.SiLU()
            
        )
        
        # mu and log std head
        
        self.mu = nn.Linear(head_4, action_dim)
        self.log_std = nn.Linear(head_4, action_dim)
        
    def forward(self, state):
        
        # state to hyper
        
        hyper = self.hyper.forward(state)
        
        # input from hyper to policy
        
        x = self.policy(hyper)
        
        # mu  and log std
        
        mu = self.mu(x)
        log_std = self.log_std(x)
        log_std = log_std.clamp(-10, 2)
        std = torch.exp(log_std)
        
        dist = torch.distributions.Normal(mu, std)
        z = dist.rsample()
        tanh_z = torch.tanh(z)
        action = tanh_z * self.max_action
        
        log_prob = dist.log_prob(z)
        log_prob = log_prob - (1 - tanh_z.pow(2) + 1e-6).log()
        log_prob = log_prob.sum(dim = -1, keepdim = True)
        
        return action, log_prob
        

### S E T U P 

In [ ]:
POLICY_NETWORK = policy_net().to(device)

print(POLICY_NETWORK)


policy_net(
  (hyper): hyper_x(
    (hyper): Sequential(
      (0): Linear(in_features=2, out_features=32, bias=True)
      (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (2): SiLU()
      (3): Linear(in_features=32, out_features=64, bias=True)
      (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (5): SiLU()
      (6): Linear(in_features=64, out_features=64, bias=True)
      (7): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (8): SiLU()
      (9): Linear(in_features=64, out_features=32, bias=True)
      (10): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (11): SiLU()
    )
  )
  (policy): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (5): SiLU()
    (6): Linear(in_features=64, out_features=32, bias=Tr

### O P T I M I Z E R 

In [ ]:
# lr

meta_lr = 1e-4
meta_iteration = 50

# optimizer

OPTIMIZER = optim.AdamW(POLICY_NETWORK.parameters(), meta_lr, weight_decay = 0)
SCHEDULER = optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max = meta_iteration, eta_min = 1e-5)


### B U F F E R

In [ ]:
class roller_buffer:
    
    def __init__(self):
        
        self.buffer = []
        
    def add(self, state, action, log_prob, reward, next_state):
    
        # add to buffer
        
        self.buffer.append({
            
            'states': safe_tensor(state),
            'actions': safe_tensor(action),
            'log_probs': safe_tensor(log_prob),
            'rewards': safe_tensor(reward),
            'next_states': safe_tensor(next_state)
        })
        
    def safe_stack(self, x):
        
        return torch.stack(x).to(device)
        
    def sample(self):
        
        # take trajectories
        
        states = [i['states'] for i in self.buffer]
        actions = [i['actions'] for i in self.buffer]
        log_probs = [i['log_probs'] for i in self.buffer]
        rewards = [i['rewards'] for i in self.buffer]
        next_states = [i['next_states'] for i in self.buffer]
        
        # form batch
        
        batch = {
            
            'states': self.safe_stack(states),
            'actions': self.safe_stack(actions),
            'log_probs': self.safe_stack(log_probs),
            'rewards': self.safe_stack(rewards),
            'next_states': self.safe_stack(next_states)
            
        }
        
        return batch
    
    def clear(self):
        
        self.buffer.clear()


### S E T U P 

In [ ]:
buffer = roller_buffer()


### C O L L E C T - T R A J

In [14]:
class collect_traj:
    
    def __init__(self, buffer = buffer, env = env, POLICY_NETWORK = POLICY_NETWORK):
        
        self.buffer = buffer
        self.env = env
        self.policy = POLICY_NETWORK
        
    def run(self, steps):
            
        obs = self.env.reset()
            
        obs = safe_tensor(obs).unsqueeze(0)
            
        total_reward = 0.0
            
        for _ in range(steps):
                
            action, log_prob = self.policy.forward(obs)
                
            action_np = action.detach().cpu().numpy()[0]
                
            next_obs, reward, done, _ = self.env.step(action_np)
                
            total_reward += reward
                
            next_obs = safe_tensor(next_obs).unsqueeze(0)
                
            self.buffer.add(obs.squeeze(0), action.squeeze(0), log_prob.squeeze(0), reward, next_obs.squeeze(0))
            
            obs = next_obs
                
            if done:
                    
                break
                

### S E T U P 

In [ ]:
COLLECT_TRAJECTORY = collect_traj()


### A D A P T - P O L I C Y

In [ ]:
def adapt_policy(learning_rate, POLICY_NETWORK = POLICY_NETWORK, buffer = buffer):
    
    # clone the policy
    
    adapted_policy = copy.deepcopy(POLICY_NETWORK)
    
    # Get batch from buffer
    
    batch = buffer.sample()
    
    states = batch['states']
    actions = batch['actions']
    rewards = batch['rewards']
    
    rewards = rewards.unsqueeze(1)
    
    # compute log probs from cloned policy
    
    _, log_probs = adapted_policy.forward(states)
    
    # compute reinforce loss
    
    loss = - (log_probs.squeeze() * rewards.squeeze()).mean()
    
    # compute gradients
    
    grad = torch.autograd.grad(loss, adapted_policy.parameters(), create_graph = True)
    
    # gradient step
    
    for p, g in zip(adapted_policy.parameters(), grad):
        
        p.data = p.data - learning_rate * g
        
    return adapted_policy


### C O L L E C T - V A L 

In [ ]:
def collect_val(adapted_policy, task, steps, env = env):
      
    env.set_task(task)
        
    obs = env.reset()
    obs = safe_tensor(obs).unsqueeze(0)
        
    total_loss = 0.0
    
    for _ in range(steps):
            
        action, log_prob = adapted_policy.forward(obs)
            
        action_np = action.detach().cpu().numpy()[0]
            
        next_obs, reward, done, _ = env.step(action_np)
            
        next_obs = safe_tensor(next_obs).unsqueeze(0)
            
        reward = safe_tensor(reward)
            
        log_prob = log_prob.squeeze()
            
        total_loss += ( - log_prob * reward)
            
        obs = next_obs
            
        if done:
                
            break
            
    val_loss = total_loss / steps
    return val_loss 


### T R A I N I N G - L O O P

In [18]:
def meta_runner_loop(num_tasks, steps, learning_rate, OPTIMIZER = OPTIMIZER, SCHEDULER = SCHEDULER, meta_iteration = meta_iteration):
    
    for iteration in tqdm(range(1, meta_iteration + 1), desc = 'FOMAML'):
        
        OPTIMIZER.zero_grad()
        total_val_loss = 0.0
        
        tasks = env.sample_task(num_tasks)
        
        for task_num in tasks:
            
            env.set_task(task_num)
            
            buffer.clear()
            
            COLLECT_TRAJECTORY.run(steps = steps)
            
            adapted_policy = adapt_policy(learning_rate)
            
            val_loss = collect_val(adapted_policy, task_num, steps)
            
            val_loss.backward()
            torch.nn.utils.clip_grad_norm_(POLICY_NETWORK.parameters(), max_norm = 0.5)
            
            total_val_loss += val_loss.item()
            
        OPTIMIZER.step()
        SCHEDULER.step()
        
        avg_val_loss = total_val_loss / num_tasks
        
        writer.add_scalar('VALIDATION Loss', avg_val_loss, iteration)
        
        print(f'epoch: {iteration} | avg_val_loss: {avg_val_loss:.4f}')
            

### S E T U P 

In [ ]:
num_tasks = 5
steps = 64
learning_rate = 1e-5

meta_runner_loop(num_tasks, steps, learning_rate)


FOMAML:   2%|▏         | 1/50 [00:03<03:00,  3.69s/it]

epoch: 1 | avg_val_loss: -0.0005


FOMAML:   4%|▍         | 2/50 [00:06<02:32,  3.18s/it]

epoch: 2 | avg_val_loss: -0.0052


FOMAML:   6%|▌         | 3/50 [00:10<02:39,  3.40s/it]

...

epoch: 48 | avg_val_loss: -0.0023


FOMAML:  98%|█████████▊| 49/50 [02:01<00:02,  2.16s/it]

epoch: 49 | avg_val_loss: -0.0025


FOMAML: 100%|██████████| 50/50 [02:03<00:00,  2.47s/it]

epoch: 50 | avg_val_loss: -0.0038
